imports

In [1]:
import os
import json
import torch 

from myhelpers.dataset_normalization import dataset_normalization
from HGNN.train.configParser import ConfigParser
from HGNN.train import dataLoader
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
experimentName="biology_paper_augmentation_effect"
experimentHash="091351b87556c0d52115d41316e73819c38ea32f4790877f20fc5137"
dataPath="/data/BGNN_data/"

cuda = 6

cuda

In [3]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

using cuda 6


Load dataset

In [4]:
# Get experiment params.
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experiment_params = None
for params_ in config_parser.getExperiments():
    if experimentHash == TrialStatistics.getTrialName(params_):
        experiment_params = params_
        break
if experiment_params is None:
    print("Could not find experiment!", experimentHash)
    raise
    
print(experiment_params)

# Get the dataset
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
datasetManager.updateParams(config_parser.fixPaths({**experiment_params, **{"img_res": None}})) # This will load images in full res but then calculate the norm in 224 pixels.
dataset = datasetManager.getDataset()

{'image_path': 'INHS_cropped', 'suffix': 'biology_paper_200max', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 128, 'n_epochs': 500, 'learning_rate': 5e-05, 'numOfTrials': 3, 'patience': 10, 'fc_width': 200, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.6, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'augmented': True, 'weight_decay': 0.0001, 'img_res': 448, 'tl_freeze': False, 'cnn_layers': 0, 'cnn_channels': 128, 'pretrained': True, 'two_nets': True, 'link_layer': 'layer3', 'dataset_norm': True, 'aug_profile': 'withRotation_PCA1.5'}
Creating dataset...


Loading images: 100%|██████████| 6695/6695 [05:42<00:00, 19.54it/s, fileName=/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588.jpg]  

Creating dataset... Done.


Calculate the normalization

In [5]:
augmentation, normalization, pad = dataset.toggle_image_loading(augmentation=False, normalization=False, pad=False)
normalizer = dataset_normalization(dataset, res=experiment_params['img_res']).getTransform()
dataset.toggle_image_loading(augmentation, normalization)

Transforming images:  21%|██▏       | 1431/6695 [00:27<02:03, 42.75it/s]


Transforming images:  42%|████▏     | 2828/6695 [00:54<01:18, 48.96it/s]


Transforming images:  63%|██████▎   | 4226/6695 [01:21<00:44, 55.97it/s]


Transforming images:  85%|████████▌ | 5709/6695 [01:49<00:23, 42.12it/s]


getting statistics: 100%|██████████| 6695/6695 [02:18<00:00, 48.17it/s]   

dataset has a mean: tensor([0.7729, 0.7427, 0.6692], device='cuda:6') and std: tensor([0.2073, 0.2598, 0.3146], device='cuda:6')


(False, False, False)

Save normalization to a file

In [6]:
if torch.cuda.is_available():
    j = json.dumps({
    "mean": normalizer[0].mean.cpu().numpy().tolist(),
    "std": normalizer[0].std.cpu().numpy().tolist(),
               })
else:
    j = json.dumps({
        "mean": normalizer[0].mean.numpy().tolist(),
        "std": normalizer[0].std.numpy().tolist(),
                   })
f = open(os.path.join(dataPath, experiment_params['image_path'], experiment_params['suffix'], 'normalization_params.json'),"w")        
f.write(j)
f.close() 